In [1]:
import pandas as pd
import pkg.dbs.postgres
from pkg import config
from pkg import claude


In [2]:
conf = config.getConfig("./config/config.ini")
pconf = conf['postgres']
psql = pkg.dbs.postgres.PSQL(pconf['host'], int(pconf['port']),pconf['db'],pconf['user'],pconf['pw'])

In [3]:
units = pd.read_csv('./data/부대.csv')
pe = pd.read_csv('./data/pe.csv',encoding='cp949')
pi = pd.read_csv('./data/인적정보.csv')
cert = pd.read_csv('./data/자격증.csv')
job = pd.read_csv('./data/직업.csv',encoding='cp949')
si = pd.read_csv('./data/si.csv',encoding='cp949')
spec = pd.read_csv('./data/특기.csv')

In [4]:
units.dtypes

ID                 object
Name               object
Commander          object
SubordinateUnit    object
Organization       object
Num                 int64
dtype: object

In [6]:
for idx in range(len(units)):
    data = units.loc[idx]
    psql.insert_data('units',data.ID,data.Commander,data.SubordinateUnit.split(','),data.Organization
                     ,data.Num, data.Name)

In [7]:
spec.dtypes

ID                object
ClassCode          int64
ClassName         object
SpecialtyName     object
Unnamed: 4       float64
dtype: object

In [8]:
for idx in range(len(spec)):
    data = spec.loc[idx]
    psql.insert_data('spec',data.ID,data.ClassName,data.ClassCode,
                     data.SpecialtyName)

In [9]:
cert.dtypes

ID      object
Name    object
dtype: object

In [10]:
for idx in range(len(cert)):
    data = cert.loc[idx]
    psql.insert_data('certificate',data.ID,data.Name)

In [11]:
job.dtypes

ID      object
Name    object
dtype: object

In [12]:
for idx in range(len(job)):
    data = job.loc[idx]
    psql.insert_data('jobcategory',data.ID,data.Name)

In [4]:
si.dtypes

ID             object
EnDate         object
DurationDay     int64
Rank           object
Specialty      object
Affiliation    object
Discipline     object
dtype: object

In [5]:
si = si.fillna('')

In [6]:
for idx in range(len(si)):
    data = si.loc[idx]
    psql.insert_data('soldierinformation',data.ID,data.EnDate,data.DurationDay,data.Rank,data.Specialty,data.Affiliation,data.Discipline.split(','))

In [10]:
pi.dtypes

ID                  object
Name                object
ResidentIDB          int64
ResidentIDF          int64
Address1            object
Address2            object
Address3            object
Address4            object
ParentsFather         bool
ParentsMother         bool
Marry                 bool
Child                int64
Religion             int64
AcademicAbility      int64
JobName             object
JobYear            float64
Certificate         object
dtype: object

In [11]:
rel = {'기독교', '불교', '천주교', '원불교', '무교', '기타'}
for i, val in enumerate(rel):
    pi.Religion = pi.Religion.replace(to_replace=i+1,value=val)

In [12]:
aa = {'중', '고', '학사', '석사', '박사'}
for i, val in enumerate(aa):
    pi.AcademicAbility = pi.AcademicAbility.replace(to_replace=i+1,value=val)

In [13]:
pi.JobYear = pi.JobYear.fillna(0)
pi = pi.fillna('')

In [14]:
pi.isna().sum()

ID                 0
Name               0
ResidentIDB        0
ResidentIDF        0
Address1           0
Address2           0
Address3           0
Address4           0
ParentsFather      0
ParentsMother      0
Marry              0
Child              0
Religion           0
AcademicAbility    0
JobName            0
JobYear            0
Certificate        0
dtype: int64

In [15]:
for idx in range(len(pi)):
    data = pi.loc[idx]
    psql.insert_data('personalinformation',data.ID,data.Name,data.ResidentIDB,data.ResidentIDF,data.Address1,data.Address2,data.Address3
                     ,data.Address4,data.ParentsFather,data.ParentsMother,data.Marry,data.Child,data.Religion,data.AcademicAbility,data.JobName,data.JobYear,data.Certificate)

In [6]:
pe.dtypes

ID               object
BloodType        object
ExamDate         object
ExamLocation     object
Height          float64
Chest           float64
Weight          float64
BpHigh          float64
BpLow           float64
VisionL         float64
VisionR         float64
PhyGrade          int64
dtype: object

In [5]:
pe.ExamDate = pe.ExamDate.astype(str)

In [7]:
pe.ExamDate = pe.ExamDate.apply(lambda x: '0' + x if len(x)<=5 else x)

In [8]:
pe.ExamDate = pe.ExamDate.apply(lambda x: x[:2]+x[2:4]+'28' if x[2:]=='0229' or '0230' else x)

In [9]:
pe.ExamDate = pe.ExamDate.apply(lambda x : '20'+x[:2]+'-'+x[2:4]+'-'+x[-2:])

In [10]:
pe

,ID,BloodType,ExamDate,ExamLocation,Height,Chest,Weight,BpHigh,BpLow,VisionL,VisionR,PhyGrade
0,23-75463585,O,2022-01-28,1검사소,186.0,101.1,86.5,104.9,73.5,0.2,0.5,3
1,23-72596660,B,2022-02-28,5검사소,181.8,95.3,83.4,117.4,79.1,1.5,1.8,1
2,23-72347837,B,2022-05-28,9검사소,176.0,100.6,80.9,93.5,73.6,0.6,0.6,2
3,23-71795363,O,2022-02-28,27검사소,168.6,95.8,78.2,94.3,67.0,1.0,1.0,1
4,24-71758837,B,2023-08-28,1검사소,187.8,95.6,92.5,109.7,64.4,1.4,1.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6995,13-13653,AB,2012-04-28,16검사소,188.5,102.1,72.3,109.3,72.9,1.6,1.8,3
6996,09-18587,A,2008-06-28,17검사소,171.3,99.8,76.0,96.7,75.5,0.6,0.7,1
6997,20-15814,A,2019-08-28,11검사소,186.8,96.8,87.4,115.9,72.3,0.0,0.2,1
6998,07-17232,B,2006-08-28,17검사소,170.4,98.9,85.3,102.5,61.1,0.1,0.2,1


In [11]:
for idx in range(len(pe)):
    data = pe.loc[idx]
    # print(data)
    psql.insert_data('pe',data.ID,data.BloodType,data.ExamDate,data.ExamLocation,data.Height,data.Chest,data.Weight,data.BpHigh,data.BpLow,data.VisionL,data.VisionR,data.PhyGrade)